Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers including convert model, generate input and performance test.

# Prerequisites

Pull dockers from Azure. It should take several minutes.

In [2]:
# !sh build.sh # For Linux
!build.sh # For Windows

Install the onnxpipeline SDK

In [1]:
import onnxpipeline

pipeline = onnxpipeline.Pipeline('model')

# onnx
#pipeline = onnxpipeline.Pipeline('onnx')

# tensorflow
#pipeline = onnxpipeline.Pipeline('mnist/model')

# pytorch
#pipeline = onnxpipeline.Pipeline('pytorch')

# cntk 
#pipeline = onnxpipeline.Pipeline('cntk')

# keras
#pipeline = onnxpipeline.Pipeline('KerasToONNX')

# sklearn
#pipeline = onnxpipeline.Pipeline('sklearn')

# caffe
#pipeline = onnxpipeline.Pipeline('caffe')

# current directory
#pipeline = onnxpipeline.Pipeline()

# test mxnet fail
#pipeline = onnxpipeline.Pipeline('mxnet')


## Run parameters

(1) local_directory: string

    Required. The path of local directory where would be mounted to the docker. All operations will be executed from this path.

(2) mount_path: string

    Optional. The path where the local_directory will be mounted in the docker. Default is "/mnt/model".

(3) print_logs: boolean

    Optional. Whether print the logs from the docker. Default is True.
    
(4) convert_directory: string

    Optional. The directory path for converting model. Default is test/.    

(5) convert_name: string

    Optional. The model name for converting model. Default is model.onnx.
    


# Config information for ONNX pipeline

In [2]:
pipeline.config()

-----------config----------------
           Container information: <docker.client.DockerClient object at 0x05E08450>
 Local directory path for volume: C:\Users\t-chuche\Documents\GitHub\onnx-pipeline\notebook/model
Volume directory path in dockers: /mnt/model
                     Result path: result
        Converted directory path: test
        Converted model filename: model.onnx
            Converted model path: test/model.onnx
        Print logs in the docker: True


# Convert model to ONNX

This image is used to convert model from major model frameworks to onnx. Supported frameworks are - caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.


You can run the docker image with customized parameters.

In [3]:

model = pipeline.convert_model(model='mnist.onnx', model_type='onnx')

# test tensorflow
#model = pipeline.convert_model(model_type='tensorflow')

# test pytorch
#model = pipeline.convert_model(model_type='pytorch', model='saved_model.pb', model_input_shapes='(1,3,224,224)')

# test cntk
#model = pipeline.convert_model(model_type='cntk', model='ResNet50_ImageNet_Caffe.model')

# test keras
#model = pipeline.convert_model(model_type='keras', model='keras_Average_ImageNet.keras')

# test sklearn
#model = pipeline.convert_model(model_type='scikit-learn', model='sklearn_svc.joblib', initial_types=("float_input", "FloatTensorType([1,4])"))

# test caffe
#model = pipeline.convert_model(model_type='caffe', model='bvlc_alexnet.caffemodel')

# test mxnet
#model = pipeline.convert_model(model_type='mxnet', model='resnet.json', model_params='resnet.params', model_input_shapes='(1,3,224,224)')



-------------

Model Conversion



Input model is already ONNX model. Skipping conversion.



-------------

MODEL INPUT GENERATION(if needed)



Input.pb already exists. Skipping dummy input generation. 



-------------

MODEL CORRECTNESS VERIFICATION





Check the ONNX model for validity 

The ONNX model is valid.



Check ONNX model for correctness. 

The original model is already onnx. Skipping correctness test. 



-------------

MODEL CONVERSION SUMMARY (.json file generated at /mnt/model/test/output.json )



{'conversion_status': 'SUCCESS',

 'correctness_verified': 'SKIPPED',

 'error_message': '',

 'input_folder': '/mnt/model/test/test_data_set_0',

 'output_onnx_path': '/mnt/model/test/model.onnx'}



## Run parameters

(1) model: string

    Required. The path of the model that needs to be converted.
    
    **IMPORTANT Only support the model path which is under the mounting directory (while initialization by Pipeline()).

(2) output_onnx_path: string

    Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

    Required. The name of original model framework. Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs_names: string

    (tensorflow) Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs_names: string

    (tensorflow) Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string 

    (mxnet) Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple 

    (pytorch, mxnet) Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: String

    Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to '7'.
    
(9) caffe_model_prototxt: string

    (caffe) Optional. The filename of deploy prototxt for the caffe madel. 

(10) initial_types: tuple (string, string)

    (scikit-learn) Optional. A tuple consist two strings. The first is data type and the second is the size of tensor type e.g., ('float_input', 'FloatTensorType([1,4])')

(11) input_json: string

    Optional. Use JSON file as input parameters.
    
    **IMPORTANT Only support the path which is under the mounting directory (while initialization by Pipeline()).
    
(12) model_inputs_names: string

    (tensorflow) Optional.
    
(13) model_outputs_names: string

    (tensorflow) Optional.    


# Performance test tool

You can run perf_test using command python perf_test.py [Your model path] [Output path on the docker]. You can use the same arguments as for onnxruntime_pert_test tool, e.g. -m for mode, -e to specify execution provider etc. By default it will try all providers available.

In [4]:
result = pipeline.perf_test(model=model)
#result = pipeline.perf_test()   # is ok, too

/home/ziyl/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] /home/ziyl/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:91 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=32646 ; hostname=5306725f2124 ; expr=cudaSetDevice(device_id_); 

Stacktrace:



Stacktrace:



Setting thread pool size to 1

Setting thread pool size to 0

Total time cost:0.0307762

Total iterations:20

Average time cost:1.53881 ms

Setting thread pool size to 0

Total time cost:0.0243878

Total iterations:20

Average time cost:1.21939 ms

Setting thread pool size to 0

Total time cost:0.0238097

Total iterations:20

Average time cost:1.19049 ms

Setting thread pool size to 1

Total time cost:0.159269

Total

# Run parameters

(1) model: string

    Required. The path of the model that wants to be performed.
    
(2) result: string

    Optional. The path of the result.
    
(3) config: string (choices=["Debug", "MinSizeRel", "Release", "RelWithDebInfo"])

    Optional. Configuration to run. Default is "RelWithDebInfo".
    
(4) mode: string (choices=["duration", "times"])

    Optional. Specifies the test mode. Value could be 'duration' or 'times'. Default is "times".

(5) execution_provider: string (choices=["cpu", "cuda", "mkldnn"])

    Optional. help="Specifies the provider 'cpu','cuda','mkldnn'. Default is ''.
    
(6) repeated_times: integer

    Optional. Specifies the repeated times if running in 'times' test mode. Default:20.
    
(7) duration_times: integer

    Optional. Specifies the seconds to run for 'duration' mode. Default:10.
    
(8) parallel: boolean

    Optional. Use parallel executor, default (without -x): sequential executor.
    
(9) threadpool_size: integer

    Optional. Threadpool size if parallel executor (--parallel) is enabled. Default is the number of cores.
    
(10) num_threads: integer

    Optional. OMP_NUM_THREADS value.
    
(11) top_n: integer

    Optional. Show percentiles for top n runs. Default:5.
    
(12) runtime: boolean

    Optional. Use this boolean flag to enable GPU if you have one.
    
(13) input_json: string

    Optional. Use JSON file as input parameters.
   

In [5]:
pipeline.print_performance(result)
#pipeline.print_result() # is ok, too

cpu_openmp 0.0013846 s

cpu_openmp_OMP_WAIT_POLICY_active 0.00139207 s

mkldnn_openmp_1_threads 0.001406933 s

mkldnn_openmp_1_threads_OMP_WAIT_POLICY_active 0.001443092 s

cpu_1_threads 0.001448245 s

cuda_1_threads error

ngraph_1_threads error

tensorrt_1_threads error

mklml_1_threads error

cuda error

ngraph error

tensorrt error

mklml error



# Result Visualization

After performance test, there would be a directory for results. 

This libray use Pandas.read_json to visualize JSON file. (orient is changeable.)

"latency.json" contains the raw data of results ordered by the average time. 

Use .latency to obtain the original latency JSON; Use .profiling to obtain original top 5 profiling JSON.

In [6]:
r = pipeline.get_result(result)
#r.latency
#r.profiling
print(result)

C:\Users\t-chuche\Documents\GitHub\onnx-pipeline\notebook/model/result


# Print latency.json

Provide parameters for top 5 performace. Use the parameter "top" to visualize more results.

In [7]:
r.prints()

,name,avg,p90,p95,cpu_usage,gpu_usage,memory_util,code_snippet
0,cpu_openmp,0.001385,0.001964,0.002447,0.000000,0,0.000000,"OrderedDict([('execution_provider', ''), ('env..."
1,cpu_openmp_OMP_WAIT_POLICY_active,0.001392,0.001825,0.002018,0.000000,0,0.000000,"OrderedDict([('execution_provider', ''), ('env..."
2,mkldnn_openmp_1_threads,0.001407,0.001725,0.001858,0.873848,0,0.341467,"OrderedDict([('execution_provider', ''), ('env..."
3,mkldnn_openmp_1_threads_OMP_WAIT_POLICY_active,0.001443,0.001790,0.001881,0.865487,0,0.340511,"OrderedDict([('execution_provider', ''), ('env..."
4,cpu_1_threads,0.001448,0.001811,0.001947,0.916762,0,0.340514,"OrderedDict([('execution_provider', ''), ('env..."


# Print profiling.json

Only provide profiling JSON for top 5 performace by giving certain index of the result. The file name is profile_[name].json
(1) index: integer
    Required. The index for top 5 profiling files.
(2) top: integer
    The number for top Ops.


In [8]:
r.print_profiling(index=4, top=5)

,cat,dur,name,op_name,ph,pid,provider,tid,ts
0,Session,73070,model_loading_uri,NaN,X,136,NaN,136,3
1,Session,6711,session_initialization,NaN,X,136,NaN,136,73081
2,Node,1,sequential/reshape/Shape_fence_before,Shape,X,136,NaN,136,84778
3,Node,61,sequential/reshape/Shape_kernel_time,Shape,X,136,CPUExecutionProvider,136,84784
4,Node,0,sequential/reshape/Shape_fence_after,Shape,X,136,NaN,136,84848


# Get environment_variables in code_snippet

Given the index from latency.json, output the environment variables.

In [9]:
r.print_environment(3)

,0
LD_LIBRARY_PATH,/perf_test/bin/RelWithDebInfo/mkldnn_openmp
OMP_WAIT_POLICY,active


# Get codes

Given the index from latency.json, output the codes.

In [10]:
print(r.get_code(3))


import onnxruntime as ort
so = rt.SessionOptions()
so.set_graph_optimization_level(2)
so.enable_sequential_execution = True
so.session_thread_pool_size(0)
session = rt.Session("/mnt/model/test/model.onnx", so)



# netron

In [ ]:
# only workable for notebook in the local server 
import netron
netron.start(model) # 'model.onnx'
from IPython.display import IFrame
IFrame('http://localhost:8080', width=700, height=350)